# Options_Trading_3


### Dummy data

In [56]:
from client.info_messages import PriceBook, PriceVolume

pb_a = PriceBook()
pb_a.instrument_id = 'AAPL'
pb_a.bids = [PriceVolume(101, 50), PriceVolume(104, 50), PriceVolume(120, 50),PriceVolume(110, 50)]
pb_a.asks = [PriceVolume(101, 100), PriceVolume(100, 200), PriceVolume(105, 70), PriceVolume(90, 250)]

pb_b = PriceBook()
pb_b.instrument_id = 'AAPL'
pb_b.bids = [PriceVolume(102, 180), PriceVolume(101, 100)]
pb_b.asks = [PriceVolume(104, 25)]

cost = 0.2
#################################################################################
def sort_ask(pb):
    result=None
    if len(pb.asks)==0:
            return result
    sort_ask = sorted(pb.asks, key=lambda x: x.price)[:3]
    return sort_ask

def sort_bid(pb):
    result=None
    if len(pb.bids)==0:
            return result
    sort_bid = sorted(pb.bids, key=lambda x: x.price, reverse=True)[:3]
    return sort_bid

####################################################################################

def best_ask(pb):
    result=None
    #we have to check if there are any asks  hasattr(pb, 'asks')
    if hasattr(pb, 'asks'):
        if len(pb.asks)==0:
            return result
        else:
            minimum_ask=pb.asks[0]
            for ask in pb.asks:
#             print(ask.price)
                if ask.price<minimum_ask.price:
                    minimum_ask=ask
            result=minimum_ask
    return result
################################################################

################################################################
def best_bid(pb):
    result=None
    if hasattr(pb, 'bids'):
        if len(pb.bids)==0:
            return result
        else:
            maximum_bid=pb.bids[0]
            for bid in pb.bids:
#             print(bid.price)
                if bid.price>maximum_bid.price:
                    maximum_bid=bid
            result=maximum_bid
    return result

bid1=best_bid(pb_a)
ask2=best_ask(pb_b)
bid2=best_bid(pb_b)
ask1=best_ask(pb_a) 



ranked_asks1=sort_ask(pb_a)
ranked_bids1=sort_bid(pb_a)
ranked_asks2=sort_ask(pb_b)
ranked_bids2=sort_bid(pb_b)


### Arbitrage

Your function is given two pricebooks (pb_a, pb_b). Both pricebooks are for the same instrument but each represents the order book on a different exchange.
An arbitrage opportunity exists when it is possible to sell or buy on one exchange and do the opposite trade on another exchange profitably.
Arbitrage is the situation where you can do a riskless trade that has positive non-zero profit

Write a function that takes two pricebooks as input and returns a boolean that represents whether an arbitrage opportunity exists between the two pricebooks.

input: PriceBook, PriceBook

output: bool (True if there is an arbitrage opportunity, False otherwise)

In [57]:
def has_arbitrage(pb_a, pb_b):
    bid1=best_bid(pb_a)
    ask2=best_ask(pb_b)
    bid2=best_bid(pb_b)
    ask1=best_ask(pb_a)
    
    
    if (bid1==None) and (ask2==None) and (bid2==None) and (ask1==None):
        result = False
    elif (bid1.price > ask2.price):
        result = True
    elif (ask1.price < bid2.price):
        result = True
    else:
        result = False
        
    return result
    
has_arbitrage(pb_a, pb_b)        

True

### Arbitrage with costs

You are again given two pricebooks, but now your are also given a cost. This cost represent the fee per lot you have to pay each exchange to trade.
Your assignment is the same as before but now you have to account for the cost to trade in determining wheter there is an arbitrage opportunity.
If you see an opportunity that could give you 10 profit but the cost is 6, it is not an arbitrage opportunity as you need to pay each exchange 6 with a total cost of 12.

Write a function that takes two pricebooks and a cost as input and returns a boolean that represents whether an arbitrage opportunity exists between the two pricebooks.

input: PriceBook, PriceBook, cost

output: bool (True if there is an arbitrage opportunity, False otherwise)

In [58]:
def has_arbitrage_with_costs(pb_a, pb_b, cost):
    bid1=best_bid(pb_a)
    ask2=best_ask(pb_b)
    bid2=best_bid(pb_b)
    ask1=best_ask(pb_a)
    
    final_cost = 2*cost
    
    volume = 0
    if (bid1==None) and (ask2==None) and (bid2==None) and (ask1==None):
        result = False
##########################################################
    elif (bid1.price > ask2.price):
        
        if (ask2.volume < bid1.volume):
            volume = ask2.volume
        elif (ask2.volume > bid1.volume):
            volume = bid1.volume
        elif (ask2.volume == bid1.volume):
            volume = bid1.volume
        volume
        pnl = (volume * (bid1.price-ask2.price))
        if (pnl > (final_cost*volume)):
            result = True
        elif (pnl <= (final_cost*volume)):
            result = False
    
        result
        
###########################################################
    elif (ask1.price < bid2.price):
        
        if (bid2.volume > ask1.volume):
            volume = ask1.volume
        elif (bid2.volume < ask1.volume):
            volume = bid2.volume
        elif (bid2.volume == ask1.volume):
            volume = bid2.volume
        volume
        pnl = (volume * (bid2.price-ask1.price))
        if (pnl > (final_cost*volume)):
            result = True
        elif (pnl <= (final_cost*volume)):
            result = False
        result
############################################################        
    else:
        result = False
        
    return result
    


has_arbitrage_with_costs(pb_a, pb_b, cost)

True

### Arbitrage lots


As before, your function is again given two pricebooks.

Write a function that calculates the maximum number of lots you can trade under arbitrage.
I.e. what is the maximum number of lots you can trade if your were to trade on all arbitrage opportunities.

input: PriceBook, PriceBook

output: lots (0 if there is no arbitrage opportunity, the maximum number of tradeable lots otherwise)

In [60]:
def arbitrage_lots(pb_a, pb_b):
    
    bid1=best_bid(pb_a)
    ask2=best_ask(pb_b)
    bid2=best_bid(pb_b)
    ask1=best_ask(pb_a)
     
    ranked_asks1=sort_ask(pb_a)
    ranked_bids1=sort_bid(pb_a)
    ranked_asks2=sort_ask(pb_b)
    ranked_bids2=sort_bid(pb_b)
    
    
    volume = 0
    if (bid1==None) and (ask2==None) and (bid2==None) and (ask1==None):
        return volume
    if bid2!=None and ask1!=None:
        if(ask1.price < bid2.price):
            for bid in ranked_bids2:
                if bid.price>ask1.price:
                    if ask1.volume<=bid.volume:
                        volume = ask1.volume
                    elif ask1.volume>bid.volume:
                        for bid in ranked_bids2:
                            if bid.price>ask1.price and bid.volume >= (ask1.volume-bid2.volume):
                                volume = ask1.volume
                            elif bid.price>ask1.price and bid.volume < (ask1.volume-bid2.volume):
                                volume = bid2.volume + bid.volume
                            else:
                                volume = bid2.volume
                    return volume
    if bid1!=None and ask2!=None:
        if (bid1.price > ask2.price):
            for bid in ranked_bids1:   
                if bid.price> ask2.price:
                    if ask2.volume<=bid.volume:
                        volume = ask2.volume
                    elif ask2.volume>bid.volume:
                        for bid in ranked_bids1:
                            if bid.price>ask2.price and bid.volume >= (ask2.volume - bid1.volume):
                                volume = ask2.volume
                            elif bid.price>ask2.price and bid.volume < (ask2.volume - bid1.volume):
                                volume = bid1.volume + bid.volume
                            else:
                                volume = bid1.volume
                    return volume
    
    return volume
        
arbitrage_lots(pb_a, pb_b)

250